In [82]:
import pandas 
from langchain.document_loaders import  DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

In [83]:
##Extract the data from pdf
def load_data_from_pdf(data):
    loaded_data = DirectoryLoader(data , glob="*.pdf" , loader_cls=PyPDFLoader ) # type: ignore

    return loaded_data.load()

In [84]:
responce = load_data_from_pdf("Data/")##only passed Directory
responce

[Document(metadata={'producer': 'Skia/PDF m133', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36', 'creationdate': '2025-03-06T11:42:43+00:00', 'title': 'Olympic Games - Wikipedia', 'moddate': '2025-03-06T11:42:43+00:00', 'source': 'Data\\Olympic Games.pdf', 'total_pages': 30, 'page': 0, 'page_label': '1'}, page_content="Olympic Games\nJeux olympiques (French)\nAbbreviation Olympiad (for summer games\nonly)\nMotto Faster, Higher, Stronger –\nTogether (Latin: Citius, Altius,\nFortius – Communiter; French:\nPlus vite, Plus haut, Plus fort –\nEnsemble)\nFirst event Summer\nAthens, Greece in 1896\nWinter\nChamonix, France in 1924\nOccur every Four years, alternated two\nyears during the four-year cycle\nHeadquartersLausanne, Switzerland\nPresident Thomas Bach\nOlympic Games\nThe modern Olympic Games (Olympics; French:\nJeux olympiques)[a][1] are the world's leading\ninternational sporting events. They feature summe

## Chunk size is the maximum number of characters in a chunk, and chunk overlap is the number of characters shared between chunks

In [85]:
##Performiing Chunking Operation
def text_split(extracted_recponce):
    text_split = RecursiveCharacterTextSplitter(chunk_size = 900, chunk_overlap = 20)
    return text_split.split_documents(extracted_recponce)

In [86]:
text_chunks = text_split(extracted_recponce=responce)
text_chunks

[Document(metadata={'producer': 'Skia/PDF m133', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36', 'creationdate': '2025-03-06T11:42:43+00:00', 'title': 'Olympic Games - Wikipedia', 'moddate': '2025-03-06T11:42:43+00:00', 'source': 'Data\\Olympic Games.pdf', 'total_pages': 30, 'page': 0, 'page_label': '1'}, page_content="Olympic Games\nJeux olympiques (French)\nAbbreviation Olympiad (for summer games\nonly)\nMotto Faster, Higher, Stronger –\nTogether (Latin: Citius, Altius,\nFortius – Communiter; French:\nPlus vite, Plus haut, Plus fort –\nEnsemble)\nFirst event Summer\nAthens, Greece in 1896\nWinter\nChamonix, France in 1924\nOccur every Four years, alternated two\nyears during the four-year cycle\nHeadquartersLausanne, Switzerland\nPresident Thomas Bach\nOlympic Games\nThe modern Olympic Games (Olympics; French:\nJeux olympiques)[a][1] are the world's leading\ninternational sporting events. They feature summe

In [87]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
def download_model_locally():
    embadding_model = HuggingFaceBgeEmbeddings(model_name = "all-MiniLM-L6-v2")

    return embadding_model

In [88]:
embedding_model = download_model_locally()

In [89]:
print(len(text_chunks))

215


In [90]:
embedding_model.embed_query("Deep Kothari")
##Convert into vector (1,384) dimention vector 

[-0.07013469934463501,
 0.09234455972909927,
 0.004564275033771992,
 0.023249851539731026,
 -0.018195118755102158,
 0.06414834409952164,
 0.009600931778550148,
 -0.04245398938655853,
 0.02383732981979847,
 -0.02015417441725731,
 0.012253514491021633,
 -0.07611887902021408,
 -0.04561899974942207,
 -0.05496469512581825,
 0.022877048701047897,
 -0.03763960301876068,
 -0.027561666443943977,
 0.05716715008020401,
 -0.08124978095293045,
 -0.015502840280532837,
 0.05731235444545746,
 0.024947835132479668,
 -0.005987153854221106,
 -0.04897452890872955,
 -0.002860600594431162,
 0.002988210180774331,
 0.04989536851644516,
 -0.09380751848220825,
 0.08190292119979858,
 -0.03518407791852951,
 -0.02852490544319153,
 0.056323353201150894,
 -0.005419235210865736,
 0.030439836904406548,
 -0.03475556522607803,
 0.09509708732366562,
 -0.045290663838386536,
 0.07343414425849915,
 0.11949387937784195,
 -0.03999210521578789,
 0.017996512353420258,
 0.018433375284075737,
 -0.0038449589628726244,
 0.015120055

In [91]:
##Fetch Api ket from secret.py
from dotenv import load_dotenv
load_dotenv()

True

In [92]:
import os
PINE_CONE_API = os.getenv("PINE_CONE_API")
os.environ['PINE_CONE_API'] = PINE_CONE_API  # type: ignore

In [93]:
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key= PINE_CONE_API)
pc

In [94]:
pc.create_index(
    name= "olympicdata", ##indexname
    dimension=384,
    metric='cosine',

    spec= ServerlessSpec(
        cloud='aws',
        region= "us-east-1"
    )
)##creatting DataBase on Pinecon.co vector database 

{
    "name": "olympicdata",
    "metric": "cosine",
    "host": "olympicdata-4bsy3b3.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [95]:
os.environ['PINECONE_API_KEY'] = PINE_CONE_API # type: ignore

In [96]:
##Now convert Chunks Into Vector and storee in VectorDataBase
from langchain_pinecone import PineconeVectorStore

databse = PineconeVectorStore.from_documents(
    # api_key = os.environ['PINE_CONE_API'],
    documents=text_chunks , ###we passed the chunkssss inside  
    index_name = "olympicdata", ### we storeewe the that chunks as vector into vector database  
    embedding = embedding_model ##passing the model that convet chunks into verctors 

)
databse

In [97]:
retriver = databse.as_retriever(search_type = "similarity" , search_kwargs = {"k" : 2})
retriver

VectorStoreRetriever(tags=['PineconeVectorStore', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x000001EB48F1A350>, search_kwargs={'k': 2})

In [101]:
retriver.invoke("What is Olympic Games")[0].page_content

'The Games have grown to the point that nearly every nation is now represented; colonies and\noverseas territories are often allowed to field their own teams. This growth has created numerous\nchallenges and controversies, including boycotts, doping, match fixing, bribery, and terrorism.\nEvery two years, the Olympics and its media exposure provide athletes with the chance to attain\nnational and international fame. The Games also provide an opportunity for the host city and\ncountry to showcase themselves to the world.\nThe Olympic Games have become a significant global event, fostering international cooperation\nand cultural exchange.[6] At the same time, hosting the Olympic Games can also bring significant\neconomic benefits and challenges to the host city, affecting infrastructure, tourism and local\ncommunities.[7]\nThe Ancient Olympic Games (Ancient Greek: τὰ Ὀ λύμπια, ta'